In [1]:
import scanpy as sc
import pandas as pd

In [2]:
meta_list = []

with open("../raw/file.txt") as fi:
    for line in fi:
        fn, name = line.strip().split("\t")
        print(name)
        meta_list.append(sc.read_h5ad("../raw/" + fn).obs)

columns = set()
for m in meta_list:
    columns |= set(m.columns)

meta = pd.concat(meta_list, join="inner")
assert set(meta.columns) == columns

print(columns)

Supercluster: Non-neuronal cells
Supercluster: MGE-derived interneurons
Supercluster: Deep layer (non-IT) excitatory neurons
Supercluster: CGE-derived interneurons
Supercluster: IT-projecting excitatory neurons
{'nCount_RNA', 'tissue_ontology_term_id', 'tissue_type', 'Subregion', 'donor_id', 'CrossArea_cluster', 'suspension_type', 'CrossArea_subclass', 'WithinArea_cluster', 'disease', 'development_stage', 'tissue', 'is_primary_data', 'self_reported_ethnicity', 'Class', 'nFeature_RNA', 'observation_joinid', 'Layer', 'cell_type', 'development_stage_ontology_term_id', 'assay', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'Location', 'sex', 'WithinArea_subclass', 'Source', 'self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id', 'disease_ontology_term_id', 'organism_ontology_term_id', 'Region', 'organism'}


In [3]:
# TODO: filer if necessary, e.g., organism or species or tissue or age (<= 18)

In [4]:
SAMPLE_HEADER = "" # TODO, e.g., "sample_id" if not available, leave it blank
EXTRACT_COLUMN = ["assay", "donor_id", "tissue_type", "disease", "organism", "sex", "tissue", "self_reported_ethnicity", "development_stage", "Region"] # TODO, e.g., ["assay", "donor_id", "tissue_type", "disease", "organism", "sex", "tissue", "self_reported_ethnicity", "development_stage"]

def get_sample_id(m): # TODO, e.g., return m["donor_id"].astype(str) + "_" + m["assay"].astype(str)
    return m["donor_id"].astype(str) + "_" + m["Region"].astype(str)
    #raise NotImplementedError


In [5]:
# Do not modify this block
meta = meta[EXTRACT_COLUMN].drop_duplicates()

if SAMPLE_HEADER:
    assert SAMPLE_HEADER in EXTRACT_COLUMN

if SAMPLE_HEADER:
    assert len(meta) == meta[SAMPLE_HEADER].nunique()
    meta["SampleID"] = meta[SAMPLE_HEADER]
else:
    if len(meta) == meta["donor_id"].nunique():
        meta["SampleID"] = meta["donor_id"]
    else:
        meta["SampleID"] = get_sample_id(meta)
        assert len(meta) == meta["SampleID"].nunique()

print(f"{len(meta)} samples")

37 samples


In [6]:
# TODO: load external metadata if necessary, e.g., from SI file
data_from_SI = pd.read_excel("../paper/Supplementary_Tables/Table_S2_cortical_area_locations.xlsx", sheet_name="Donor_Metadata")
data_from_SI
data_from_SI = data_from_SI.rename(columns={"Donor ID":"donor_id"})
data_from_SI

meta = meta.merge(data_from_SI, on="donor_id", how="left")

In [8]:
meta["DonorID"] = meta["donor_id"]
meta["Organism"] = "Homo sapiens"
meta["Sex"] = meta["sex"]
meta['Sex'] = meta['Sex'].replace('male', 'Male')
meta['Sex'] = meta['Sex'].replace('female', 'Female')
meta["Race"] = ''
meta["Ethnicity"] = meta["self_reported_ethnicity"]
meta["Ethnicity"] = meta["Ethnicity"].replace("unknown", "Unknown")
meta["Tissue"] = "Brain"
meta['Region'] = meta['tissue'].str.capitalize()
meta["Disease"] = meta["disease"]
meta["Disease"] = meta["disease"].replace("normal", "Normal")
meta["Type"] = "Postmortem"
meta["Assay"] = meta["assay"]

meta["Age group"] = meta["development_stage"]
meta["PMI"] = meta["Postmortem Interval (hr)"]
meta["Cause of death (presumed)"] = meta["Presumed Cause of Death"]
meta["RIN"] = meta["Average RIN"]



meta_stan = meta[["SampleID", "DonorID", "Organism", "Age", "Sex", "Race", "Ethnicity", "Tissue", "Region", "Disease", "Type", "Assay"]]
meta_orig = meta[["SampleID","Age group", "PMI","Cause of death (presumed)", "RIN"]]

meta_stan.to_csv("../metadata/sample_standard.tsv", sep="\t", index=False)
meta_orig.to_csv("../metadata/sample_original.tsv", sep="\t", index=False)

/tmp/ipykernel_86427/4132932400.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  meta['Sex'] = meta['Sex'].replace('male', 'Male')
/tmp/ipykernel_86427/4132932400.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  meta['Sex'] = meta['Sex'].replace('female', 'Female')
/tmp/ipykernel_86427/4132932400.py:8: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
 

In [9]:
meta_stan

,SampleID,DonorID,Organism,Age,Sex,Race,Ethnicity,Tissue,Region,Disease,Type,Assay
0,H18.30.002_V1,H18.30.002,Homo sapiens,50,Male,,Unknown,Brain,Primary visual cortex,Normal,Postmortem,10x 3' v3
1,H18.30.001_V1,H18.30.001,Homo sapiens,60,Female,,Unknown,Brain,Primary visual cortex,Normal,Postmortem,10x 3' v3
2,H200.1023_V1,H200.1023,Homo sapiens,43,Female,,Unknown,Brain,Primary visual cortex,Normal,Postmortem,10x 3' v3
3,H19.30.002_V1,H19.30.002,Homo sapiens,29,Male,,Unknown,Brain,Primary visual cortex,Normal,Postmortem,10x 3' v3
4,H19.30.001_V1,H19.30.001,Homo sapiens,42,Male,,Unknown,Brain,Primary visual cortex,Normal,Postmortem,10x 3' v3
5,H18.30.002_A1,H18.30.002,Homo sapiens,50,Male,,Unknown,Brain,Primary auditory cortex,Normal,Postmortem,10x 3' v3
6,H18.30.001_A1,H18.30.001,Homo sapiens,60,Female,,Unknown,Brain,Primary auditory cortex,Normal,Postmortem,10x 3' v3
7,H200.1023_A1,H200.1023,Homo sapiens,43,Female,,Unknown,Brain,Primary auditory cortex,Normal,Postmortem,10x 3' v3
8,H19.30.001_A1,H19.30.001,Homo sapiens,42,Male,,Unknown,Brain,Primary auditory cortex,Normal,Postmortem,10x 3' v3
9,H19.30.002_A1,H19.30.002,Homo sapiens,29,Male,,Unknown,Brain,Primary auditory cortex,Normal,Postmortem,10x 3' v3
